# Client Data Only Model clientmodel1a 

- feature selection on PenFed data only
- without LTV application feature
- train test split: 
  "train": {"start_date": "2021-07-01", "end_date": "2022-07-01"},
  "valid": {"start_date": "2022-07-01", "end_date": "2023-01-01"},
  "test": {"start_date": "2023-01-01", "end_date": "2024-01-01"}

In [1]:
# House keeping settings
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [2]:
import os
import pandas as pd
import json
import numpy as np

from model_engine.assets.utils import load_asset
from model_engine.io.loaders import load_json
from model_engine.power.post_sale import PowerModelBuilder
from model_engine.analysis.suggesters import evaluate_datesplits
import warnings

from zaml.common.utils import load_state

import model_engine, zaml
model_engine.__version__, zaml.__version__

/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/zamlexplain/utils_compile.py:23: NumbaPendingDeprecationWarning: The 'pycc' module is pending deprecation. Replacement technology is being developed.

Pending Deprecation in Numba 0.57.0. For more information please see: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-the-numba-pycc-module
  from numba.pycc import CC
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/lyt/.conda/envs/penfed_modeling/lib/python3.1

('v1.13.0', '34.5.2')

In [3]:
from model_engine.model_builder.asset_parser import asset_parser
from model_engine.model_builder import ModelBuilder

In [4]:
input_model_id = 'clientmodel1a'
input_target_name = 'final_DQ90_m12'


final_model_output_path = f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{input_model_id}'


print(f'Final model path: {final_model_output_path}')

Final model path: /d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel1a


In [5]:
asset = json.load(open(f'./{input_model_id}_asset.json'))

In [6]:
asset['config']['data_split'] 

{'train': {'start_date': '2021-07-01', 'end_date': '2022-07-01'},
 'valid': {'start_date': '2022-07-01', 'end_date': '2023-01-01'},
 'test': {'start_date': '2023-01-01', 'end_date': '2024-01-01'}}

In [7]:
model_builder = ModelBuilder(asset)
model_builder.configure_run(final_model_output_path, 
                            cache=True) 

state = asset_parser(asset)

input_asset = (json.dumps(state['input_asset'], indent=4))
print(input_asset)

{
    "data": {
        "trade": {
            "asset": "experian/arf7/trade.json",
            "data": "/d/shared/silver_projects_v2/project_power/shared_data/processed/client_data/penfed_autoindirect/trade.parquet",
            "io_params": {
                "drop_duplicates": true,
                "memory_efficient": true
            }
        },
        "inq": {
            "asset": "experian/arf7/inquiry.json",
            "data": "/d/shared/silver_projects_v2/project_power/shared_data/processed/client_data/penfed_autoindirect/inq.parquet",
            "io_params": {
                "drop_duplicates": true,
                "memory_efficient": true
            }
        },
        "collec": {
            "asset": "experian/arf7/collections.json",
            "data": "/d/shared/silver_projects_v2/project_power/shared_data/processed/client_data/penfed_autoindirect/collec.parquet",
            "io_params": {
                "drop_duplicates": true,
                "memory_efficient": 

In [8]:
with open (os.path.join(final_model_output_path, 'input_configuration.json'), 'w') as outfile:
    outfile.write(input_asset)

In [9]:
from model_engine.model_builder import build_model

output = build_model.build_model(asset, final_model_output_path)

parsing model-builder asset
Configuring model builder


INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_asset>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_data>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <monotonic_constraints_list>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <data_split>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <train_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <train_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <valid_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <valid_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <test_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <test_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engin

building model


INFO:zaml.artifact_engine.logger:Finished <versions>, total time spent: 0:00:05.264574
INFO:zaml.artifact_engine.logger:Executing MonotonicConstraintsListParser <parsed_monotonic_constraints_list>...
INFO:zaml.artifact_engine.logger:Finished <parsed_monotonic_constraints_list>, total time spent: 0:00:00.315996
INFO:zaml.artifact_engine.logger:Executing SplitterArtifact <splitter>...
INFO:zaml.artifact_engine.logger:Finished <splitter>, total time spent: 0:00:00.000598
INFO:zaml.artifact_engine.logger:Executing DataArtifact <data>...
INFO:zaml.artifact_engine.logger:Finished <data>, total time spent: 0:00:00.000288
INFO:zaml.artifact_engine.logger:Executing ExclusionListParser <parsed_exclusion_list>...
INFO:zaml.artifact_engine.logger:Finished <parsed_exclusion_list>, total time spent: 0:00:00.000268
INFO:zaml.artifact_engine.logger:Executing BivariateFeParser <parsed_biv_fe_instructions>...
INFO:zaml.artifact_engine.logger:Finished <parsed_biv_fe_instructions>, total time spent: 0:00:

-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.025s
-------------------------
Name: bnkr
Transformer type: None
Number of features: 9
Time spent: 0.020s
-------------------------
Name: client_data
Transformer type: None
Number of features: 180
Time spent: 0.020s
-------------------------
Name: collec
Transformer type: None
Number of features: 6
Time spent: 0.021s
-------------------------
Name: inq
Transformer type: None
Number of features: 18
Time spent: 0.021s
-------------------------
Name: member
Transformer type: None
Number of features: 29
Time spent: 0.028s
-------------------------
Name: trade
Transformer type: None
Number of features: 8465
Time spent: 0.067s
-------------------------
Name: app FE
Transformer type: OneToOneEngine
Number of features: 0
Time spent: 0.066s
-------------------------
Name: bnkr FE
Transformer type: ManyToOneEngine
Number of features: 8
Time spent: 0.062s
-------------------------
Name: client_data FE


...

INFO:zaml.artifact_engine.logger:Finished <valid_fe_data>, total time spent: 0:01:39.891032
INFO:zaml.artifact_engine.logger:Executing FeDataArtifact <test_fe_data>...


-------------------------
Name: CorrelationFeatureSelection
Transformer type: CorrelationFeatureSelection
Number of features: 4053
Time spent: 0.238s
-------------------------
Name: GainFeatureSelection
Transformer type: GainFeatureSelection
Number of features: 190
Time spent: 0.036s
-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.000s
-------------------------
Name: bnkr
Transformer type: None
Number of features: 9
Time spent: 0.000s
-------------------------
Name: client_data
Transformer type: None
Number of features: 180
Time spent: 0.000s
-------------------------
Name: collec
Transformer type: None
Number of features: 6
Time spent: 0.000s
-------------------------
Name: inq
Transformer type: None
Number of features: 18
Time spent: 0.000s
-------------------------
Name: member
Transformer type: None
Number of features: 29
Time spent: 0.000s
-------------------------
Name: trade
Transformer type: None
Number of features: 8465
Time spen

INFO:zaml.artifact_engine.logger:Finished <test_fe_data>, total time spent: 0:01:49.225470
INFO:zaml.artifact_engine.logger:Executing StaticAssetArtifact <static_asset>...


-------------------------
Name: CorrelationFeatureSelection
Transformer type: CorrelationFeatureSelection
Number of features: 4053
Time spent: 0.552s
-------------------------
Name: GainFeatureSelection
Transformer type: GainFeatureSelection
Number of features: 190
Time spent: 0.053s


INFO:zaml.artifact_engine.logger:Finished <static_asset>, total time spent: 0:00:03.945764
INFO:zaml.artifact_engine.logger:Executing TrainHistoryArtifact <train_history>...
INFO:zaml.artifact_engine.logger:Finished <train_history>, total time spent: 0:00:00.000338
INFO:zaml.artifact_engine.logger:Executing BestModelParamsArtifact <best_model_params>...
INFO:zaml.artifact_engine.logger:Finished <best_model_params>, total time spent: 0:00:00.000278
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <train_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_scores>, total time spent: 0:00:00.503702
INFO:zaml.artifact_engine.logger:Executing SubmodelScoresArtifact <train_submodel_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_submodel_scores>, total time spent: 0:00:00.000322
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <valid_scores>...
INFO:zaml.artifact_engine.logger:Finished <valid_scores>, total time spent: 0:00:00.124544
INFO:zaml.artifact_en

In [10]:
scoreFiles = []

for j in ['_auc.json','_ks.json']:
#     for i in ['train', 'valid', 'test']:
    for i in ['train','test',]:
        scoreFiles.append(i+j)
        
scoreFiles_model = {}
for i in scoreFiles:
    name = i.split('.')[0]
    # print(name)
    scoreFiles_model[name] = json.load(open(os.path.join(final_model_output_path, i), 'rb'))


for i in scoreFiles_model:
    print(i, scoreFiles_model[i])

train_auc {'auc': 0.7817485471126692}
test_auc {'auc': 0.7583506731489998}
train_ks {'ks': 0.42237928909472067}
test_ks {'ks': 0.39659051923571137}


In [12]:
features_importance = pd.read_parquet(os.path.join(final_model_output_path, 'feature_importance.parquet'))
features_importance.shape

(190, 2)

In [13]:
features_importance.head(20)

,feature,importance
0,client_data_LTV_NORM,0.009366
1,trade_clAmt__mean_by_ecoa_individual,0.002611
2,inq_inquiries__count_by_indCode_auto_mortgage,0.002165
3,trade_months_since_lstPmtDate__min_by_prtfType...,0.002108
4,trade_months_since_lstPmtDate__min_by_acctType...,0.001974
5,trade_hcAmt__sum,0.001949
6,trade_clAmt_accts_with_rcntPmt__min_by_acctTyp...,0.001707
7,trade_months_since_openDate__mean_by_acctType_...,0.001690
8,trade_blnc_to_hc_accts_with_rcntPmt__mean_by_a...,0.001586
9,trade_clAmt_active_accts__mean_by_ecoa_individual,0.001566


In [14]:
final_model_output_path

'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel1a'

In [1]:
!ls /d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel1a

artifact_manifest.json			test_scores.parquet
asset.json				test_target.parquet
best_model_params.json			test_zest_scores.parquet
calibration_object.obj			top_features.parquet
feature_definition.parquet		train_app.parquet
feature_importance.parquet		train_auc.json
fit_time_info.json			train_data_summary.json
input_configuration.json		train_fe_data.parquet
keep_features.json			train_history.json
keep_features_nationalmodel.json	train_ks.json
key_factors_mapping.json		train_scores.parquet
model.obj				train_target.parquet
model_strategy.json			train_zest_scores.parquet
mrm_pipeline.obj			valid_app.parquet
parsed_monotonic_constraints_list.json	valid_auc.json
pipeline.obj				valid_data_summary.json
score_recalibration_mapping.json	valid_fe_data.parquet
splitter.obj				valid_ks.json
static_asset.json			valid_scores.parquet
test_app.parquet			valid_target.parquet
test_auc.json				valid_zest_scores.parquet
test_data_summary.json			value_based_key_factor_mapping.json
test_fe_data.parquet			ver